In [1]:
import os

In [2]:
os.chdir(r"C:\Users\Surya Teja\OneDrive\Desktop\text-summarizer-project\Text-Summarizer-Project")

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list

In [4]:
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
        )

        return data_validation_config

In [6]:
import os

In [7]:
from src.textSummarizer.logging import logger

In [11]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            # Ensure the validation directory exists
            create_directories([self.config.root_dir])

            dataset_dir = os.path.join("artifacts", "data_ingestion", "samsum_dataset")

            if not os.path.exists(dataset_dir):
                validation_status = False
                msg = f"Dataset directory not found: {dataset_dir}"
                os.makedirs(os.path.dirname(self.config.STATUS_FILE), exist_ok=True)
                with open(self.config.STATUS_FILE, 'w') as f:
                    f.write(msg)
                logger.info(msg)
                return False

            # Only consider directory names that match required items
            present_items = set([name for name in os.listdir(dataset_dir)])
            required_items = set(self.config.ALL_REQUIRED_FILES)

            missing = required_items - present_items
            validation_status = len(missing) == 0

            os.makedirs(os.path.dirname(self.config.STATUS_FILE), exist_ok=True)
            with open(self.config.STATUS_FILE, 'w') as f:
                if validation_status:
                    f.write("Validation status: True\nAll required files/directories are present.")
                    logger.info("All required files/directories are present.")
                else:
                    f.write(f"Validation status: False\nMissing: {sorted(list(missing))}")
                    logger.warning(f"Validation failed. Missing: {sorted(list(missing))}")

            return validation_status

        except Exception as e:
            raise e


In [12]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    result = data_validation.validate_all_files_exist()
    print(f"Validation result: {result}")
except Exception as e:
    raise e


[2025-11-02 15:25:23,259] [INFO] [textSummarizerLogger] - yaml file: config\config.yaml loaded successfully
[2025-11-02 15:25:23,260] [INFO] [textSummarizerLogger] - yaml file: params.yaml loaded successfully
[2025-11-02 15:25:23,261] [INFO] [textSummarizerLogger] - created directory at: artifacts
[2025-11-02 15:25:23,262] [INFO] [textSummarizerLogger] - created directory at: artifacts\data_validation
[2025-11-02 15:25:23,264] [INFO] [textSummarizerLogger] - All required files/directories are present.
Validation result: True
[2025-11-02 15:25:23,260] [INFO] [textSummarizerLogger] - yaml file: params.yaml loaded successfully
[2025-11-02 15:25:23,261] [INFO] [textSummarizerLogger] - created directory at: artifacts
[2025-11-02 15:25:23,262] [INFO] [textSummarizerLogger] - created directory at: artifacts\data_validation
[2025-11-02 15:25:23,264] [INFO] [textSummarizerLogger] - All required files/directories are present.
Validation result: True
